效果完全没有区别，只是实现方式不一样。

shuffle=True 的实现方式： 在每个 epoch 开始时将整个数据集打乱，然后按照打乱后的顺序划分 batch。再按照batch_size 个数依次提取数据 

sampler.BatchSampler(random_sampler) 的实现方式：（数据不会打乱）

step 1、RandomSampler 会生成随机的索引。

step 2、BatchSampler 根据上面随机出来的索引生成 batch 组。

step 3、拿着每个batch 组的索引去取 数据

相同点：

每个epoch 都会重新打乱

都不会重复采样，除非你通过参数指定了可以重复采样  

其他说明：

shuffle=True 的性能更高一些，而 BatchSampler灵活性更高，因为你可以通过 BatchSampler 设计更复杂的采样方式

在 Dataloader 中使用 batch_sampler 的常见目的之一，是为了兼容 DistributedSampler，比如：

In [ ]:
if args.distributed:
    sampler_train = DistributedSampler(dataset_train)
    sampler_val = DistributedSampler(dataset_val, shuffle=False)
else:
    sampler_train = torch.utils.data.RandomSampler(dataset_train)
    sampler_val = torch.utils.data.SequentialSampler(dataset_val)

batch_sampler_train = torch.utils.data.BatchSampler(sampler_train, args.batch_size, drop_last=True)

data_loader_train = DataLoader(dataset_train,
                               batch_sampler=batch_sampler_train,
                               collate_fn=utils.collate_fn,
                               )
data_loader_val = DataLoader(dataset_val,
                             args.batch_size,
                             sampler=sampler_val,
                             drop_last=False,
                             collate_fn=utils.collate_fn,
                             )

跑个小例子，看一下

In [ ]:
import torch
import torch.utils.data.sampler as sampler
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self):
        self.data = [1, 2, 3, 4, 5]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]


dataset = MyDataset()

# =============================================
random_sampler = sampler.RandomSampler(data_source=dataset)
batch_sampler = sampler.BatchSampler(random_sampler, batch_size=2, drop_last=False)
dataloader1 = DataLoader(dataset, batch_sampler=batch_sampler)

for epoch in range(3):
    for index, data in enumerate(dataloader1):
        print(index, data)
print('*'*30)

# =============================================
dataloader2 = DataLoader(dataset, batch_size=2, shuffle=True)

for epoch in range(3):
    for index, data in enumerate(dataloader2):
        print(index, data)